# 1. import libraries

In [1]:
import io
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pandas.tseries.offsets import DateOffset
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use('seaborn-white')

# 2. data 1

In [3]:
df1 = pd.read_csv('Ecommerce_Data.csv')
df1['year'] = pd.DatetimeIndex(df1['Date']).year.astype(str)
df1['week1'] = pd.DatetimeIndex(df1['Date']).week.astype(str)
df1['week2'] = np.where(df1['week1'].str.len()==1, '0'+df1['week1'], df1['week1'])
df1.shape

(81601, 13)

In [4]:
df1['date'] = df1['year'].astype(str)+"-"+df1['week2'].astype(str)
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']
df1 = df1[['date', 'Country', 'Quantity', 'TotalPrice']]
df1 = df1.groupby(['date','Country']).agg({'Quantity':'sum','TotalPrice':'sum'}).reset_index()
df1.shape

(672, 4)

# 3. data 2

In [5]:
df2 = pd.read_csv('US_Holiday_Dates_(2004-2021).csv')
df2 = df2[df2['Year'].isin([2010,2011])]
df2 = df2.sort_values('Date')
df2['year'] = pd.DatetimeIndex(df2['Date']).year.astype(str)
df2['week1'] = pd.DatetimeIndex(df2['Date']).week.astype(str)
df2['week1'] = np.where((df2['Month']==1) & (df2['week1']=='52'), '1', df2['week1'])
df2['week2'] = np.where(df2['week1'].str.len()==1, '0'+df2['week1'], df2['week1'])
df2.shape

(38, 9)

In [6]:
df2['date'] = df2['year'].astype(str)+"-"+df2['week2'].astype(str)
df2['year'] = df2['year'].astype(int)
df2['week1'] = df2['week1'].astype(int)
df2['week1'] = np.where((df2['WeekDay'].isin(['Monday','Tuesday'])) & (df2['week1']==1), 52, 
                        np.where(df2['WeekDay'].isin(['Monday','Tuesday']), df2['week1']-1, df2['week1']))
df2['week1'] = np.where((df2['WeekDay'].isin(['Saturday','Sunday'])) & (df2['week1']==52), 1, 
                        np.where(df2['WeekDay'].isin(['Saturday','Sunday']), df2['week1']+1, df2['week1']))
df2.shape

(38, 10)

In [7]:
df2['year'] = np.where((df2['WeekDay'].isin(['Monday','Tuesday'])) & (df2['week1']==1), df2['year']-1, df2['year'])
df2['year'] = np.where((df2['WeekDay'].isin(['Saturday','Sunday'])) & (df2['week1']==52), df2['year']+1, df2['year'])
df2['week1'] = df2['week1'].astype(str)
df2['week2'] = np.where(df2['week1'].str.len()==1, '0'+df2['week1'], df2['week1'])
df2['also'] = df2['year'].astype(str)+"-"+df2['week2'].astype(str)
df2 = df2[['date', 'also', 'Holiday']]
df3 = df2[df2['date']!=df2['also']]
df3.shape

(29, 3)

In [8]:
df2 = df2[['date', 'Holiday']]
df3 = df3[['also', 'Holiday']]
df3.columns = ['date', 'Holiday']
df2 = pd.concat([df2,df3])
df2 = df2.sort_values('date')
df2.shape

(67, 2)

# 4. merged data

In [9]:
df3 = pd.merge(df1, df2, how='left', on='date')
df3['Christmas'] = np.where(df3['Holiday'].isin(['Christmas Day','Christmas Eve']),1,0)
df3['New Year'] = np.where(df3['Holiday'].isin(['New Year’s Eve','New Year\'s Day','Martin Luther King, Jr. Day']),1,0)
df3['Valentine'] = np.where(df3['Holiday'].isin(['Valentine’s Day','Washington\'s Birthday']),1,0)
df3['Easter'] = np.where(df3['Holiday'].isin(['Western Easter','Eastern Easter']),1,0)
df3['Memorial'] = np.where(df3['Holiday'].isin(['Memorial Day']),1,0)
df3['Juneteenth'] = np.where(df3['Holiday'].isin(['Juneteenth']),1,0)
df3['4thJuly'] = np.where(df3['Holiday'].isin(['4th of July']),1,0)
df3['Labor Day'] = np.where(df3['Holiday'].isin(['Labor Day Weekend','Labor Day']),1,0)
df3['Columbus Day'] = np.where(df3['Holiday'].isin(['Columbus Day']),1,0)
df3['Veterans Day'] = np.where(df3['Holiday'].isin(['Veterans Day']),1,0)
df3['Thanksgiving'] = np.where(df3['Holiday'].isin(['Thanksgiving Day','Thanksgiving Eve']),1,0)
df3.shape

(793, 16)

In [10]:
df3 = df3.drop('Holiday', axis=1)
df3 = df3.drop_duplicates()
df3.shape

(672, 15)

# 5. Export data

In [11]:
df3.to_csv('model_data.csv', index=False)
df3.shape

(672, 15)